# Fragestellungen

### Fluggesellschaften

#### Welche Fluggesellschaften haben prozentual (+/-) die grössten Abweichungen der geplanten Flugzeit zur effektiven Flugzeit (geplante Abflugzeit bis effektive Ankunftszeit)?

#### Bei welchen Fluggesellschaften werden die meisten Flüge gestrichen?

#

### Flughäfen

#### Welche 10 Flughäfen haben die kleinste und grösste Abflugverspätung (ohne cancellation)?

#### Bei welchen Flughäfen werden die meisten Flüge gestrichen?

#

### Weitere Fragestellungen

#### An welchen Tagen gibt es am meisten Flugbewegungen?

#### Können Unregelmässigkeiten wie Naturkatastrophen oder Terroranschläge festgestellt werden? 